<a href="https://colab.research.google.com/github/LuizScarparo/DataScience/blob/main/Dashboard_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!apt-get update -qq
!apt-get install -qq pandoc texlive-xetex


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('/content/database.csv', sep=',', encoding='utf-8', dtype={'Idade': str})

df['Anos'] = (
    df['Idade']
      .astype(str)
      .str.split(pat='-', n=1).str[0]
)
df['Anos'] = pd.to_numeric(df['Anos'], errors='coerce')

# converte CrtsA e Cmp%
df['CrtsA'] = pd.to_numeric(df['CrtsA'], errors='coerce')
df['Cmp%']  = pd.to_numeric(df['Cmp%'].str.replace(',','.'), errors='coerce')

# calcula Gols_per90
df['Gols_per90'] = df['Gols'] / (df['Min.'] / 90)

# 2.3 Distribuição de Nacionalidades
counts = df['Nação'].value_counts()
plt.figure(figsize=(6,6))
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Distribuição de Nacionalidades')
plt.savefig('nacionalidades.png', bbox_inches='tight')
plt.close()

# 2.4 Distribuição de Posições
counts = df['Pos.'].value_counts()
perc   = counts / counts.sum() * 100
major  = counts[perc >= 2].copy()
others = counts[perc < 2].sum()
if others > 0:
    major['Outros'] = others
plt.figure(figsize=(6,6))
plt.pie(major, labels=major.index, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Distribuição de Posições')
plt.savefig('posicoes.png', bbox_inches='tight')
plt.close()

# 2.7 Idade vs. Cartões Amarelos
data_ic = df.dropna(subset=['Anos','CrtsA'])
m_ic, b_ic = np.polyfit(data_ic['Anos'], data_ic['CrtsA'], 1)
plt.figure(figsize=(6,4))
plt.scatter(data_ic['Anos'], data_ic['CrtsA'], alpha=0.6, s=30)
plt.plot(data_ic['Anos'], m_ic*data_ic['Anos']+b_ic, '--', lw=2, color='red')
plt.title('Idade vs. Cartões Amarelos')
plt.xlabel('Anos'); plt.ylabel('Cartões Amarelos')
plt.savefig('idade_cartoes.png', bbox_inches='tight')
plt.close()

# 2.8 Top-10 Artilheiros
top10 = df.groupby('Jogador')['Gols'].sum().nlargest(10)
plt.figure(figsize=(8,6))
(top10[::-1]).plot(kind='barh')
plt.title('Top 10 Artilheiros')
plt.xlabel('Total de Gols'); plt.ylabel('')
plt.savefig('top10.png', bbox_inches='tight')
plt.close()

# 2.9 Heatmap de Correlação
cols = ['Gols','Assis.','xG','xAG','Cmp%','PrgP','Conduções']
corr = df[cols].dropna().corr()
plt.figure(figsize=(6,6))
plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar()
plt.xticks(range(len(cols)), cols, rotation=45, ha='right')
plt.yticks(range(len(cols)), cols)
plt.title('Heatmap de Correlação')
plt.savefig('heatmap.png', bbox_inches='tight')
plt.close()

# 2.10 Eficiência de Gols por 90 Minutos
data_eff = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Min.','Gols_per90'])
m_eff, b_eff = np.polyfit(data_eff['Min.'], data_eff['Gols_per90'], 1)
plt.figure(figsize=(6,4))
plt.scatter(data_eff['Min.'], data_eff['Gols_per90'], alpha=0.6, s=30)
plt.plot([0, data_eff['Min.'].max()],
         [b_eff, m_eff*data_eff['Min.'].max()+b_eff],
         '--', lw=2, color='green')
plt.title('Gols/90 min vs Minutos Jogados')
plt.xlabel('Minutos Jogados'); plt.ylabel('Gols/90 min')
plt.savefig('eficiencia.png', bbox_inches='tight')
plt.close()


In [53]:
# Cell 3: Montagem Dinâmica do `report.md` (versão organizada)

with open('report.md', 'w') as f:
    # Cabeçalho
    f.write("# Relatório Completo de Estatísticas de Jogadores\n\n")
    f.write("**Nomes:** Luiz Henrique Scarparo, Emanuel Mariuzza\n\n")

    # Tema
    f.write("## 1. Tema do Projeto\n")
    f.write("Análise exploratória de estatísticas de futebol brasileiro usando Python "
            "(pandas, matplotlib, seaborn, plotly) e construção de um dashboard interativo com Dash.\n\n")

    # Repositório
    f.write("## 2. Repositório no GitHub\n")
    f.write("[https://github.com/LuizScarparo/DataScience](https://github.com/LuizScarparo/DataScience)\n\n")

    # Dataset
    f.write("## 3. Dataset Utilizado\n")
    f.write("- **Origem:** `/content/database.csv`\n")
    f.write("- **Descrição:** Estatísticas individuais dos jogadores do campeonato brasileiro de 2015\n")
    f.write("- **Fonte dos dados:** Kaggle (eduardopalmieri/brasileiro-player-stats-2024)\n\n")
    f.write("### 3.1 Variáveis principais\n")
    f.write("| Coluna       | Descrição                                          |\n")
    f.write("|:------------:|:--------------------------------------------------|\n")
    f.write("| Jogador      | Nome completo do atleta                            |\n")
    f.write("| Time         | Clube ou seleção                                   |\n")
    f.write("| Nação        | Nacionalidade                                      |\n")
    f.write("| Pos.         | Posição (GK, DF, MF, FW, etc.)                     |\n")
    f.write("| Min.         | Minutos jogados na temporada                       |\n")
    f.write("| Gols         | Total de gols marcados                             |\n")
    f.write("| Assis.       | Total de assistências                              |\n")
    f.write("| Cmp%         | Percentual de passes completados                   |\n")
    f.write("| xG           | Expected Goals (gols esperados)                    |\n")
    f.write("| xAG          | Expected Assists (assistências esperadas)          |\n\n")

    # Modelos e Métodos
    f.write("## 4. Modelos e Métodos\n")
    f.write("- **Regressão Linear** (numpy.polyfit) para identificar tendências em:\n")
    f.write("  1. Minutos jogados × Gols por 90 minutos\n")
    f.write("  2. Idade (Anos) × Cartões Amarelos\n")
    f.write("- **Visualizações**:\n")
    f.write("  - Gráficos de pizza (nationalities e posições)\n")
    f.write("  - Scatter plots com linha de tendência\n")
    f.write("  - Heatmap de correlação entre métricas avançadas\n\n")

    # Resultados
    f.write("## 5. Resultados Obtidos\n")
    f.write("### 5.1 Top 10 Artilheiros\n")

# 5.1 Lista dinâmica de artilheiros
import pandas as pd
df = pd.read_csv('/content/database.csv', sep=',', encoding='utf-8', dtype={'Idade': str})
top10 = df.groupby('Jogador')['Gols'].sum().nlargest(10)
with open('report.md', 'a') as f:
    for idx, (jog, gols) in enumerate(top10.items(), 1):
        f.write(f"{idx}. **{jog}**: {gols} gols\n")
    f.write("\n")

# 5.2 Gráficos
sections = [
    ("Distribuição de Nacionalidades", "nacionalidades.png"),
    ("Distribuição de Posições",       "posicoes.png"),
    ("Eficiência de Gols por 90 Minutos", "eficiencia.png"),
    ("Heatmap de Correlação",          "heatmap.png"),
    ("Idade vs. Cartões Amarelos",     "idade_cartoes.png"),
]
with open('report.md', 'a') as f:
    f.write("### 5.3 Visualizações\n\n")
    for title, img in sections:
        f.write(f"#### {title}\n")
        f.write(f"![{title}]({img})\n\n")

    # Resumo dos Resultados
    f.write("## 6. Resumo dos Resultados\n")
    f.write("Em síntese, a análise revelou:\n")
    f.write("- **Nacionalidades:** Predomínio de brasileiros (>60%), com argentinos e europeus em seguida (cada <10%).\n")
    f.write("- **Posições:** Defensores e meio-campistas somam ~70%; alas, atacantes e goleiros abaixo de 2% agrupados como \"Outros\".\n")
    f.write("- **Assistências vs. Posições:** Meio-campistas têm maior mediana de assistências.\n")
    f.write("- **Gols vs. Posições:** Atacantes lideram na mediana de gols.\n")
    f.write("- **Eficiência de Gols:** Tendência positiva leve entre minutos jogados e gols/90 min.\n")
    f.write("- **Correlações Avançadas:** Forte associação entre Gols↔xG e Assistências↔xAG.\n\n")

    # Conclusões
    f.write("## 7. Conclusões\n")
    f.write("Este relatório apresentou uma visão abrangente das estatísticas individuais: "
            "identificou padrões de performance, comportamento disciplinar e a eficácia ofensiva dos jogadores. "
            "O dashboard facilita a exploração interativa destes insights.\n")



In [54]:
# converte o Markdown em PDF
!pandoc report.md \
  --from markdown+yaml_metadata_block \
  --pdf-engine=xelatex \
  -V geometry:margin=1in \
  -o report.pdf

# baixa o PDF
from google.colab import files
files.download('report.pdf')


[WARNING] Missing character: There is no ↔ (U+2194) (U+2194) in font [lmroman10-regular]:mapping=t
[WARNING] Missing character: There is no ↔ (U+2194) (U+2194) in font [lmroman10-regular]:mapping=t


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>